In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
df = pd.read_csv('model_supplier_product_remove_duplicates.csv')
df

,Maker1,Maker2,Product,jaccardSimilarity,intersection,sharedSuppliers
0,BYD,Brilliance,power_steering_pump,1.000000,9,"[Changshu Jinhua Machinery Co., Ltd., Fuxin Da..."
1,BYD,Dongfeng Passenger Vehicle,power_steering_pump,1.000000,9,"[Changshu Jinhua Machinery Co., Ltd., Fuxin Da..."
2,BYD,FAW Car,power_steering_pump,1.000000,9,"[Changshu Jinhua Machinery Co., Ltd., Fuxin Da..."
3,BYD,SAIC Passenger Vehicle,power_steering_pump,1.000000,9,"[Changshu Jinhua Machinery Co., Ltd., Fuxin Da..."
4,Brilliance,Dongfeng Passenger Vehicle,power_steering_pump,1.000000,9,"[Changshu Jinhua Machinery Co., Ltd., Fuxin Da..."
...,...,...,...,...,...,...
124,GM,Nissan,shock_absorber,0.206897,6,"[ZF Friedrichshafen AG, ZF Active Safety And E..."
125,Ford,Toyota,shock_absorber,0.205128,8,[KYB Manufacturing do Brasil Fabricante de Aut...
126,Toyota,Volkswagen,shock_absorber,0.205128,8,"[ZF Friedrichshafen AG, KYB Manufacturing do B..."
127,Nissan,Volkswagen,shock_absorber,0.200000,6,"[ZF Friedrichshafen AG, ZF Active Safety And E..."


In [80]:
df[['Maker1', 'Maker2']].value_counts()

Maker1        Maker2         
Nissan        Toyota             7
Honda         Toyota             5
Daimler       Volkswagen         4
Audi          Volkswagen         4
Ford          GM                 4
                                ..
Changan Ford  SAIC Volkswagen    1
              FAW-Volkswagen     1
Changan       SAIC GM            1
              NETA               1
Toyota        Volkswagen         1
Name: count, Length: 86, dtype: int64

In [72]:
nissan_and_toyota = df[(df['Maker1'] == 'Nissan') & (df['Maker2'] == 'Toyota')]

In [111]:
product_weights = {
    'steering_gear': 2,
    'stablizer': 0.7,
    'exhaust_manifold': 0.05,
    'abs_esc': 0.9,
    'disc_brake_(pad)': 2,
    'clutch': 0.2,
    'shock_absorber': 0.6
}

# Create a new column with the weights
nissan_and_toyota['weight'] = nissan_and_toyota['Product'].map(product_weights)

# If any products don't have defined weights, you might want to handle that
# df['weight'] = df['weight'].fillna(1.0)  # Default weight of 1.0 for undefined products

# Calculate weighted Jaccard similarity
nissan_and_toyota['weighted_jaccard'] = nissan_and_toyota['jaccardSimilarity'] * nissan_and_toyota['weight'];

In [112]:
nissan_and_toyota

,Maker1,Maker2,Product,jaccardSimilarity,intersection,sharedSuppliers,weight,weighted_jaccard
51,Nissan,Toyota,steering_gear,0.411765,7,"[JTEKT Corporation, Rongtai Industrial Develop...",2.00,0.823529
58,Nissan,Toyota,stablizer,0.400000,8,"[New Mather Metals, Inc., NHK of America Suspe...",0.70,0.280000
63,Nissan,Toyota,exhaust_manifold,0.388889,7,"[Futaba Industrial Co., Ltd., Sango Co., Ltd.,...",0.05,0.019444
74,Nissan,Toyota,abs_esc,0.350000,7,"[Bosch (Robert Bosch LLC), Aisin Corporation (...",0.90,0.315000
76,Nissan,Toyota,disc_brake_(pad),0.344828,10,"[Akebono Brake Corporation, ADVICS Co., Ltd., ...",2.00,0.689655
96,Nissan,Toyota,clutch,0.300000,6,"[ZF Friedrichshafen AG, Schaeffler AG, P.T. EX...",0.20,0.060000
103,Nissan,Toyota,shock_absorber,0.285714,10,"[KYB Americas Corporation - Indiana, thyssenkr...",0.60,0.171429


In [114]:
# Calculate the sum of all weighted similarities
total_weighted_sum = nissan_and_toyota['weighted_jaccard'].sum()

# Calculate the sum of all weights used
total_weights = nissan_and_toyota['weight'].sum()

# Normalize by dividing by the sum of weights (optional)
normalized_weighted_sum = total_weighted_sum / total_weights

print(f"Weighted sum of Jaccard similarities: {total_weighted_sum:.4f}")
print(f"Normalized weighted sum: {normalized_weighted_sum:.4f}")

Weighted sum of Jaccard similarities: 2.3591
Normalized weighted sum: 0.3657


In [ ]:
df = pd.read_csv('model_supplier_product_remove_duplicates.csv')
product_weights = {
    'steering_gear': 2,
    'stablizer': 0.7,
    'exhaust_manifold': 0.05,
    'abs_esc': 0.9,
    'disc_brake_(pad)': 2,
    'clutch': 0.2,
    'shock_absorber': 0.6
}

def maker_weighted_sim(df, maker_1, maker_2, weights: dict[str: float]):
    df = df.copy()
    maker_1 = df['Maker1'] == maker_1
    maker_2 = df['Maker2'] == maker_2

    df = df[maker_1 & maker_2]
    df['weight'] = df['Product'].map(weights)
    df['weighted_jaccard'] = df['jaccardSimilarity'] * df['weight']
    df['weight'] = df['weight'].fillna(1.0)

    total_weighted_sum = df['weighted_jaccard'].sum()

    normalized_score = total_weighted_sum / total_weights
    return normalized_score


In [108]:
maker_weighted_sim(df, 'Nissan', 'Toyota', product_weights)

0.365745364349072